## Natural hazard index for any location

Author: Germaine Pötgen

______________________________________________________________________________________

- As of now the dataset for the variable friction is corrupted, so the corresponding code is commented out
- Change the region of interest and the normalization reference according to your needs
- Comment out index variables you don't want to include in the hazard index

### Imports

In [1]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping

In [2]:
from source import *

### Authorize access to Google Earth Engine

In [3]:
# ee.Authenticate()

In [4]:
ee.Initialize()

### Define region of interest

Change the region of interest according to your needs

##### Create a geometry to use as region of interest

In [5]:
vienna_bbox = ee.Geometry.BBox(16.2678922, 48.1195168, 16.5371291, 48.2618604)
vienna_point = ee.Geometry.Point(16.363449, 48.210033)
vienna_polygon = ee.Geometry.Polygon([
    [
        [16.1808, 48.3236],  
        [16.3841, 48.3261],  
        [16.5761, 48.3090],  
        [16.5796, 48.2493],  
        [16.5110, 48.1404], 
        [16.3073, 48.0837],  
        [16.2257, 48.1304],  
        [16.1808, 48.3236]  
    ]
])

##### Upload a geojson to use as region of interest

In [6]:
# Define path to JSON file
region_path = r'C:\Users\germa\Dokumente\MasterGeographie\4_Semester\SE_Konversatorium\Notebooks\LANDESGRENZEOGD.json'

# Convert GeoJSON to Earth Engine object
region_ee_object = geemap.geojson_to_ee(region_path)

# Get geometry of GEE object
region_gee_geometry_json = region_ee_object.geometry()

##### Upload a shapefile to use as region of interest

In [7]:
# Load shapefile as geodataframe
region_gdf = gpd.read_file(r'C:\Users\germa\Dokumente\MasterGeographie\4_Semester\SE_Konversatorium\Notebooks\LANDESGRENZEOGD\LANDESGRENZEOGDPolygon.shp')

# Define function to convert shapefile geometry to GEE geometry
def shp_to_ee_geom(shp_geom):
    geojson = mapping(shp_geom) 
    return ee.Geometry(geojson)

# Extract the first row's geometry from the geodataframe, adjust as needed
region_geometry = region_gdf.loc[0, 'geometry']

# Convert shapefile polygon to GEE geometry
region_gee_geometry_shp = shp_to_ee_geom(region_geometry)

##### Visualize region of interest on map

In [8]:
Map = geemap.Map()
Map.addLayer(region_gee_geometry_shp, {'color': 'red'}, 'Vienna Polygon')
Map.centerObject(region_gee_geometry_shp)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Define normalization reference

Change the normalization reference according to your needs. Choose from countries in country_names_sorted.

##### Use Global Administrative Unit Layers (GAUL) dataset to get country borders

https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_SIMPLIFIED_500m_2015_level0

In [9]:
# Load GAUL dataset
all_countries = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level0')

In [10]:
# Display country names to choose from
country_names = all_countries.aggregate_array('ADM0_NAME').getInfo()
country_names_sorted = sorted(country_names)

print(country_names_sorted)

['Abyei', 'Afghanistan', 'Aksai Chin', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Arunachal Pradesh', 'Ashmore and Cartier Islands', 'Australia', 'Austria', 'Azerbaijan', 'Azores Islands', 'Bahamas', 'Bahrain', 'Baker Island', 'Bangladesh', 'Barbados', 'Bassas da India', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bird Island', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Canada', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'China/India', 'Christmas Island', 'Clipperton Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', "Côte d'Ivoire", "Dem 

In [11]:
# Define country
country_name = "Austria"

# Filter dataset for specified country
country = all_countries.filter(ee.Filter.eq('ADM0_NAME', country_name))

##### Create fishnet over specified country

Adjust h_interval/v_interval (horizontal/vertical intervals in degrees) or number of rows/cols according to the size of the chosen country to avoid memory errors

In [12]:
country_fishnet = geemap.fishnet(country.geometry(), h_interval=0.3, v_interval=0.3, delta=1)
# country_fishnet = geemap.fishnet(country.geometry(), rows=50, cols=50, delta=1)

# Optional: clip country_fishnet to exact border outlines -> might lead to computation timeout
country_fishnet_clip = country_fishnet.map(lambda tile: tile.intersection(country.geometry(), ee.ErrorMargin(1)))

##### Visualize normalization reference on map

In [20]:
Map = geemap.Map()
Map.addLayer(country_fishnet, {'color': 'red'}, 'Fishnet')
Map.centerObject(country_fishnet)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Calculate natural hazard index value for region of interest

##### Create list for regions of interest and list for region names

In [14]:
region_of_interest_list = [region_gee_geometry_json, region_gee_geometry_shp, vienna_bbox, vienna_polygon, vienna_point]
region_of_interest_names_list = ['vienna_json', 'vienna_shp', 'vienna_bbox', 'vienna_polygon', 'vienna_point']

##### Calculate min/max values of all variables for normalization reference

Comment out index variables you don't want to include in the hazard index

In [15]:
min_values = {}
max_values = {}
mean_values = {}

In [16]:
min_values["pdsi"], max_values["pdsi"] = pdsi_average.min_max(country_fishnet)
min_values["precip_change"], max_values["precip_change"] = precipitation_ann_change.min_max(country_fishnet)
min_values["precip_coeff_variation"], max_values["precip_coeff_variation"] = precipitation_coefficient_variation.min_max(country_fishnet)
min_values["precip_anomaly"], max_values["precip_anomaly"] = precipitation_anomaly.min_max(country_fishnet)
min_values["surface_temp"], max_values["surface_temp"] = surface_temperature_max.min_max(country_fishnet)
min_values["temp_anomaly"], max_values["temp_anomaly"] = temperature_anomaly.min_max(country_fishnet)
min_values["humidity"], max_values["humidity"] = specific_humidity.min_max(country_fishnet)
min_values["flow"], max_values["flow"] = flow_accumulation.min_max(country_fishnet)
# min_values["friction"], max_values["friction"] = friction.min_max(country_fishnet)
min_values["slope"], max_values["slope"] = slope.min_max(country_fishnet)
min_values["ssm"], max_values["ssm"], mean_values["ssm"] = surface_soil_moisture.min_max(country_fishnet)

Min PDSI: -179.37698412698413, Max PDSI: 57.6468253968254
Min precipitation accumulation change: -320.32866253256793, Max precipitation accumulation change: 374.89495741426936
Min precipitation cv: 0.06579386392234346, Max precipitation cv: 0.16971386550318923
Min maximum precipitation anomaly: 31.961308506273085, Max maximum precipitation anomaly: 273.9893998532068
Min surface temperature: 295.70001220703125, Max surface temperature: 311.7363586425781
Min temperature anomaly: 1.116546630859375, Max temperature anomaly: 2.349029541015625
Min specific humidity: 0.003416921943426132, Max specific humidity: 0.006809286307543516
Min flow: 1, Max flow: 23023383
Min slope: 0, Max slope: 85.7269515991211
Min ssm: 11.87097454071045, Max ssm: 24.642255783081055, Mean ssm: 20.61974480042773


##### Calculate normalized variable values for each region of interest

In [1]:
var_1_results = []
var_2_results = []
var_3_results = []
var_4_results = []
var_5_results = []
var_6_results = []
var_7_results = []
var_8_results = []
var_9_results = []
var_10_results = []
var_11_results = []

for region_of_interest in region_of_interest_list:
    var1 = pdsi_average.get_pdsi_region(region_of_interest, min_values["pdsi"], max_values["pdsi"])
    var_1_results.append(var1)

    var2 = precipitation_ann_change.get_precipitation_change_region(region_of_interest, min_values["precip_change"], max_values["precip_change"])
    var_2_results.append(var2)

    var3 = precipitation_coefficient_variation.get_precipitation_coefficient_variation_region(region_of_interest, min_values["precip_coeff_variation"], max_values["precip_coeff_variation"])
    var_3_results.append(var3)

    var4 = precipitation_anomaly.get_precipitation_anomaly_region(region_of_interest, min_values["precip_anomaly"], max_values["precip_anomaly"])
    var_4_results.append(var4)

    var5 = surface_temperature_max.get_surface_temperature_max_region(region_of_interest, min_values["surface_temp"], max_values["surface_temp"])
    var_5_results.append(var5)

    var6 = temperature_anomaly.get_temperature_anomaly_region(region_of_interest, min_values["temp_anomaly"], max_values["temp_anomaly"])
    var_6_results.append(var6)

    var7 = specific_humidity.get_specific_humidity_region(region_of_interest, min_values["humidity"], max_values["humidity"])
    var_7_results.append(var7)

    var8 = flow_accumulation.get_flow_accumulation_region(region_of_interest, min_values["flow"], max_values["flow"])
    var_8_results.append(var8)

    # var9 = friction.get_friction_region(region_of_interest, min_values["friction"], max_values["friction"])
    # var_9_results.append(var9)

    var10 = slope.get_slope_region(region_of_interest, min_values["slope"], max_values["slope"])
    var_10_results.append(var10)

    var11 = surface_soil_moisture.get_surface_soil_moisture_region(region_of_interest, min_values["ssm"], max_values["ssm"], mean_values["ssm"])
    var_11_results.append(var11)

##### Calculate hazard index values and save results in a csv file

In [18]:
data = {
    "Region of Interest": region_of_interest_names_list,
    "PDSI average": var_1_results,
    "precipitation change": var_2_results,
    "precipitation coefficient variation": var_3_results,
    "precipitation anomaly": var_4_results,
    "surface temperature max": var_5_results,
    "temperature anomaly": var_6_results,
    "specific humidity": var_7_results,
    "flow accumulation": var_8_results,
    #"friction": var_9_results,
    "slope": var_10_results,
    "ssm": var_11_results,
}

df = pd.DataFrame(data)

df["hazard index"] = df[[
    "PDSI average", "precipitation change", "precipitation coefficient variation",
    "precipitation anomaly", "surface temperature max", "temperature anomaly",
    "specific humidity", "flow accumulation", #"friction",
    "slope", "ssm"
]].mean(axis=1)

df

,Region of Interest,PDSI average,precipitation change,precipitation coefficient variation,precipitation anomaly,surface temperature max,temperature anomaly,specific humidity,flow accumulation,slope,ssm,hazard index
0,vienna_json,0.573214,0.216158,0.614473,0.097728,0.916131,0.185261,0.230128,0.002525,0.040145,0.754372,0.363014
1,vienna_shp,0.573214,0.216158,0.614473,0.097728,0.916131,0.185261,0.230128,0.002525,0.040145,0.754372,0.363014
2,vienna_bbox,0.569944,0.217533,0.597059,0.085973,0.923608,0.190241,0.221169,0.002869,0.026872,0.780789,0.361606
3,vienna_polygon,0.575141,0.214623,0.620931,0.100739,0.918013,0.174150,0.232417,0.001927,0.044754,0.747386,0.363008
4,vienna_point,0.579519,0.230145,0.605463,0.088911,0.916099,0.197593,0.231797,0.000000,0.018821,0.802659,0.367101


In [19]:
# Save the DataFrame as a csv file
df.to_csv('index_results.csv')